In [9]:
import tkinter as tk
from tkinter import messagebox, ttk
from tkcalendar import DateEntry
from datetime import datetime

class ToDoApp:
    def __init__(self, root):
        self.root = root
        self.root.title("To-Do List Application")
        self.root.geometry("650x600")
        self.root.configure(bg="#2e2e2e")  # Dark background color

        self.tasks = []  # List to store all tasks with status
        self.displayed_tasks = []  # List to store currently displayed tasks

        # Main Frame
        self.main_frame = tk.Frame(root, bg="#2e2e2e")
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)

        # Title
        self.title_label = tk.Label(self.main_frame, text="To-Do List", bg="#2e2e2e", fg="#ffffff", font=("Arial", 24, "bold"))
        self.title_label.pack(pady=(0, 10))

        # Task Entry and Buttons
        self.entry_frame = tk.Frame(self.main_frame, bg="#2e2e2e")
        self.entry_frame.pack(pady=(0, 10))

        self.task_entry = tk.Entry(self.entry_frame, width=30, bg="#3e3e3e", fg="#ffffff", borderwidth=2, relief="flat", font=("Arial", 14))
        self.task_entry.grid(row=0, column=0, padx=5, pady=5)

        self.date_entry = DateEntry(self.entry_frame, width=12, background='darkblue', foreground='white', borderwidth=2, font=("Arial", 12))
        self.date_entry.grid(row=0, column=1, padx=5, pady=5)

        self.time_frame = tk.Frame(self.entry_frame, bg="#2e2e2e")
        self.time_frame.grid(row=0, column=2, padx=5, pady=5)

        self.hour_entry = tk.Entry(self.time_frame, width=3, bg="#3e3e3e", fg="#ffffff", borderwidth=2, relief="flat", font=("Arial", 12))
        self.hour_entry.grid(row=0, column=0, padx=5, pady=5)
        self.hour_entry.insert(0, "HH")  # Placeholder for hour

        self.minute_entry = tk.Entry(self.time_frame, width=3, bg="#3e3e3e", fg="#ffffff", borderwidth=2, relief="flat", font=("Arial", 12))
        self.minute_entry.grid(row=0, column=1, padx=5, pady=5)
        self.minute_entry.insert(0, "MM")  # Placeholder for minute

        self.ampm_combobox = ttk.Combobox(self.time_frame, values=["AM", "PM"], state="readonly", width=5, font=("Arial", 12))
        self.ampm_combobox.grid(row=0, column=2, padx=5, pady=5)
        self.ampm_combobox.set("AM")  # Default value

        # Buttons
        self.add_button = tk.Button(self.entry_frame, text="Add", command=self.add_task, bg="#005a5a", fg="#ffffff", font=("Arial", 12, "bold"), relief="flat", padx=10, pady=5)
        self.add_button.grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)

        self.update_button = tk.Button(self.entry_frame, text="Update", command=self.update_task, bg="#007a7a", fg="#ffffff", font=("Arial", 12, "bold"), relief="flat", padx=10, pady=5)
        self.update_button.grid(row=1, column=1, padx=5, pady=5, sticky=tk.W)

        self.delete_button = tk.Button(self.entry_frame, text="Delete", command=self.delete_task, bg="#b52e2e", fg="#ffffff", font=("Arial", 12, "bold"), relief="flat", padx=10, pady=5)
        self.delete_button.grid(row=1, column=2, padx=5, pady=5, sticky=tk.W)

        self.mark_completed_button = tk.Button(self.entry_frame, text="Complete", command=self.mark_as_completed, bg="#006600", fg="#ffffff", font=("Arial", 12, "bold"), relief="flat", padx=10, pady=5)
        self.mark_completed_button.grid(row=2, column=0, padx=5, pady=5, sticky=tk.W)

        self.filter_status = tk.StringVar(value="All")
        self.filter_menu = ttk.Combobox(self.entry_frame, textvariable=self.filter_status, values=["All", "Pending", "Completed"], state="readonly", width=10)
        self.filter_menu.grid(row=2, column=1, padx=5, pady=5)

        self.filter_button = tk.Button(self.entry_frame, text="Filter", command=self.filter_tasks, bg="#005a5a", fg="#ffffff", font=("Arial", 12, "bold"), relief="flat", padx=10, pady=5)
        self.filter_button.grid(row=2, column=2, padx=5, pady=5)

        # Task Listbox with Scrollbar
        self.task_frame = tk.Frame(self.main_frame, bg="#2e2e2e")
        self.task_frame.pack(fill=tk.BOTH, expand=True)

        self.task_listbox = tk.Listbox(self.task_frame, selectmode=tk.SINGLE, width=75, height=20, bg="#3e3e3e", fg="#ffffff", selectbackground="#005a5a", selectforeground="#ffffff", font=("Arial", 12), borderwidth=0, highlightthickness=0)
        self.task_listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        self.scrollbar = tk.Scrollbar(self.task_frame)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.task_listbox.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.task_listbox.yview)

        # Bind double click to mark as completed
        self.task_listbox.bind("<Double-1>", self.on_task_double_click)

    def add_task(self):
        task = self.task_entry.get()
        date = self.date_entry.get_date()
        hour = self.hour_entry.get()
        minute = self.minute_entry.get()
        ampm = self.ampm_combobox.get()
        if task and hour.isdigit() and minute.isdigit() and ampm in ["AM", "PM"]:
            time = f"{hour}:{minute} {ampm}"
            task_text = f"{task} | Due: {date} {time} - Pending"
            self.tasks.append(task_text)
            self.displayed_tasks.append(task_text)
            self.update_listbox()
            self.task_entry.delete(0, tk.END)
            self.hour_entry.delete(0, tk.END)
            self.minute_entry.delete(0, tk.END)
            self.ampm_combobox.set("AM")  # Reset AM/PM
        else:
            messagebox.showwarning("Input Error", "Please enter a task and a valid time.")

    def update_task(self):
        selected_task_index = self.task_listbox.curselection()
        if selected_task_index:
            new_task = self.task_entry.get()
            date = self.date_entry.get_date()
            hour = self.hour_entry.get()
            minute = self.minute_entry.get()
            ampm = self.ampm_combobox.get()
            if new_task and hour.isdigit() and minute.isdigit() and ampm in ["AM", "PM"]:
                current_text = self.displayed_tasks[selected_task_index[0]]
                status = current_text.split(" - ")[1]
                new_text = f"{new_task} | Due: {date} {hour}:{minute} {ampm} - {status}"
                self.displayed_tasks[selected_task_index[0]] = new_text
                # Update the tasks list as well
                self.tasks = [new_text if task == current_text else task for task in self.tasks]
                self.update_listbox()
                self.task_entry.delete(0, tk.END)
                self.hour_entry.delete(0, tk.END)
                self.minute_entry.delete(0, tk.END)
                self.ampm_combobox.set("AM")  # Reset AM/PM
            else:
                messagebox.showwarning("Input Error", "Please enter a new task description and a valid time.")
        else:
            messagebox.showwarning("Selection Error", "Please select a task to update.")

    def delete_task(self):
        selected_task_index = self.task_listbox.curselection()
        if selected_task_index:
            task_to_delete = self.displayed_tasks[selected_task_index[0]]
            self.displayed_tasks.pop(selected_task_index[0])
            self.tasks = [task for task in self.tasks if task != task_to_delete]
            self.update_listbox()
        else:
             messagebox.showwarning("Selection Error", "Please select a task to delete.")

    def mark_as_completed(self):
        selected_task_index = self.task_listbox.curselection()
        if selected_task_index:
            current_text = self.displayed_tasks[selected_task_index[0]]
            if "Pending" in current_text:
                new_text = current_text.replace("Pending", "Completed")
                self.displayed_tasks[selected_task_index[0]] = new_text
                # Update the tasks list as well
                self.tasks = [new_text if task == current_text else task for task in self.tasks]
                self.update_listbox()
            else:
                messagebox.showinfo("Status Update", "Task is already completed.")
        else:
            messagebox.showwarning("Selection Error", "Please select a task to mark as completed.")

    def filter_tasks(self):
        filter_option = self.filter_status.get()
        if filter_option == "All":
            self.displayed_tasks = self.tasks[:]
        else:
            self.displayed_tasks = [task for task in self.tasks if filter_option in task]
        self.update_listbox()

    def update_listbox(self):
        self.task_listbox.delete(0, tk.END)
        for task in self.displayed_tasks:
            if "Completed" in task:
                self.task_listbox.insert(tk.END, task)
                self.task_listbox.itemconfig(tk.END, {'bg': '#d0ffd0', 'fg': '#004d00'})  # Light green for completed tasks
            else:
                self.task_listbox.insert(tk.END, task)
                self.task_listbox.itemconfig(tk.END, {'bg': '#f0f0f0', 'fg': '#000000'})  # Light gray for pending tasks

    def on_task_double_click(self, event):
        self.mark_as_completed()

if __name__ == "__main__":
    root = tk.Tk()
    app = ToDoApp(root)
    root.mainloop()
